# Sessão Síncrona entre Emitter e Receiver com uso de Curvas Elípticas

## Descrição do Exercício

Versão alternativa da sessão do exercício anterior mas com o uso de **curvas Elípticas**.

**Requisitos a ter em conta para esta versão de comunicação:**

- Cifra Simétrica **AES** substituída pela cifra **ChaCha20Poly1305**
- Protocolo de acordo de chaves **Diffie-Hellman** agora por Curvas Elípticas (**ECDH**)
    - Esquema de assinaturas **DSA** agora por Curvas Elípticas (**ECDSA**)
- Foi escolhida a curva elíptica *SECP384R1*

## Descrição da Implementação

Não foi necessário definir quaisquer decisões consoante o que foi pedido. Apenas se efetuaram as substituições pedidas ficando o programa similar ao anterior em termos de **Emitter** e **Receiver**.
A grande diferença foi a retirada da autenticação por **HMAC**, dado que agora a cifra pedida pelo professor já é autenticada, através do **MAC** intitulado de **Poly1305**.

## Resolução do Exercício

In [1]:
import os, io
from SyncPipe import SyncPipe
from ECDH_ECDSA import ecdh_ecdsa
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.exceptions import *

### Definição do Agente Emitter
Função que trata de definir o *Emitter* e seu envolvimento no processo de comunicação segura.

**Descrição do processo:**

- Estabelecimento do acordo de chaves *Elliptic-curve Diffie-Hellman* com assinatura *Elliptic Curve Digital Signature Algorithm*
- Criação de um *nonce* aleatório a cada comunicação
- Criação da cifra ChaCha20Poly1305

Com todos estes valores necessários e com um canal pronto para enviar os dados, a ideia é ir lendo blocos de 32 *bytes* devidamente cifrados, enviando-os sucessivamente pelo canal. Após se enviar todos os blocos de dados fecha-se o canal.

In [2]:
tamanhoMensagem = 2**10

def Emitter(connection):
    
    # Acordo de chaves DH e assinatura DSA
    key = ecdh_ecdsa(connection)
        
    # Criação dum input stream para enviar a mensagem
    inputs = io.BytesIO(bytes('D'*tamanhoMensagem,'utf-8'))
    
    # Inicialização do nonce aleatório (12 bytes)
    nonce  = os.urandom(12)
    
    # Cifra ChaCha20Poly1305
    cipher = ChaCha20Poly1305(key)
    
    # Enviar o IV para o peer
    connection.send(nonce)
    
    # Criação dum buffer para ler os blocos de 32 bytes (256 bits)    
    buffer = bytearray(32)
    
    # lê, cifra e envia sucessivos blocos do input 
    try:
        
        # Enquanto existem blocos (while)
        while inputs.readinto(buffer): 
            ciphertext = cipher.encrypt(nonce, bytes(buffer), None)
            connection.send(ciphertext)       
    except Exception as err:
        print("Erro no emissor: {0}".format(err))

    # Fechar o stream para colocar inputs
    inputs.close()
    
    # Fechar a conexão com o peer
    connection.close()

### Definição do Agente Receiver

Função que trata de definir o *Receiver* e o seu envolvimento no proceso de comunicação segura.

**Descrição do processo:**

- Estabelecimento do acordo de chaves *Elliptic-curve Diffie-Hellman* com assinatura *Elliptic Curve Digital Signature Algorithm*
- Recebimento do *nonce* enviado pelo *Emitter*
- Decifração com cifra ChaCha20Poly1305

O processo estipulado para o *Receiver* difere do agente que lhe envia a informação. Com as devidas verificações pode-se ir lendo os blocos que vão surgindo por parte do *Emitter* ao mesmo tempo que decifram. Imprime-se a mensagem final no *Notebook* em si.

In [3]:
def Receiver(connection):
    
    # Acordo de chaves DH e assinatura DSA
    key = ecdh_ecdsa(connection)
    
    # Inicializa um output stream para receber o texto decifrado
    outputs = io.BytesIO()
    
    # Recebe o Vetor IV
    nonce = connection.recv()
    
    # Cifra ChaCha20Poly1305
    cipher = ChaCha20Poly1305(key)
    
    # Operar a cifra: ler da conexão um bloco, autenticá-lo, decifrá-lo e escrever o resultado no stream de output
    try:
        while True:
            try:
                # Receber do Emitter o buffer de 32 bytes e tag MAC associada
                buffer = connection.recv()

                ciphertext = bytes(buffer)
                decifredtext = cipher.decrypt(nonce, ciphertext, None)
                    
                # Colocar no stream o texto decifrado e corretamente autenticado
                outputs.write(decifredtext)
                break
            except InvalidSignature as err:
                raise Exception("Autenticação do ciphertext ou metadados: {}".format(err))
                
        # Escrever no Jupyter Notebook os resultados colocados na stream
        print(outputs.getvalue())
        
    except Exception as err:
        print("Erro no receptor: {0}".format(err))

    # Fechar o stream dos outputs
    outputs.close()

    # Fechar a conexão
    connection.close()
    

### Criação dos Pipes e execução dos Agentes

In [4]:
SyncPipe(Emitter, Receiver).auto()

Está verificada a assinatura do parceiro.
b'DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD'
Está verificada a assinatura do parceiro.
Valid ECDH (MAC)
Valid ECDH (MAC)


## Observações Finais

- A parte do acordo de chaves ECDH com assinatura ECDSA foi feita, igualmente ao exercício anterior, num ficheiro à parte
- Maior dificuldade em entender como usar a cifra ChaCha20Poly1305, dado que se trata de uma *Stream Cipher* e estamos a lidar com uma **sessão síncrona** com uma implementação a pensar em blocos.

## Referências

- Wikipedia, Elliptic-curve Diffie–Hellman [https://en.wikipedia.org/wiki/Elliptic-curve_Diffie%E2%80%93Hellman](https://en.wikipedia.org/wiki/Elliptic-curve_Diffie%E2%80%93Hellman) (Acedido a 10 março 2020)
- Wikipedia, Elliptic Curve Digital Signature Algorithm [https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm](https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm) (Acedido a 10 março 2020)
- Doc Sagemath, Elliptic curves [http://doc.sagemath.org/html/en/constructions/elliptic_curves.html](http://doc.sagemath.org/html/en/constructions/elliptic_curves.html) (Acedido a 10 março 2020)
- Cryptography, Authenticated encryption [https://cryptography.io/en/latest/hazmat/primitives/aead/](https://cryptography.io/en/latest/hazmat/primitives/aead/) (Acedido a 10 março 2020)
- Cryptography, Elliptic curve cryptography [https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ec/](https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ec/) (Acedido a 11 março 2020)